## Measurement and performing the computation

Now that we have seen the preparation of the resource state, we are ready to perform the actual computation we want.
This is done by applying rotations and measuring in the X/Y basis. 

So in general for each qubit, we apply:
- A phase shift (i.e a P gate)
- Followed by a Hadamard
- Then a measurement in the 0/1 basis.

For the sake of simplicity let's assume, the computation we want is the identity, so no rotations are needed and we will only measure our qubits.


Now the question is, in which order do we measure our qubits? 

# Graph flow
In a graph of N columns, each column is considered a (computation) layer, the information flows from left to right. The first layer is considered to be the input layer and the last layer is the output layer.

For cluster states, we perform the measurement layer-by-layer and in each layer, we perform the measurement row-by-row from top to bottom.
Information and some other side effects of the projective measurement propagate to the neighboring qubits, this mechanism is known as feed-forward.

![Cluster feed-forward illustration](images/feed-forward.png)

(from [Cluster State Quantum Computing](https://www.cs.umd.edu/class/spring2018/cmsc457/report/Cluster_State_Quantum_Computing.pdf))


The idea here is to teleport information across the cluster. If you recall the teleportation chapter from the Qiskit textbook, you will notice that at the end of the protocol, we corrected the teleportation by applying X and Z gates selectively depending on previous measurement outcomes. MBQC also works by relying on the same teleportation concept. It is strongly suggested to check out the teleportation tutorial if you haven't yet!

In our case, we also need to apply the same correction concept which is based on what is known as Pauli stabilizers.
Simply put, Pauli stabilizers are a set of operators, each associated to one vertex of the graph, the idea is that a stabilizer $K$ should not change the graph $|G\rangle$ so
$$ K_i := X_i \otimes_ {j\in N(i)} Z_j $$
$$ K_i |G\rangle = |G\rangle $$


The Flow tells us how to apply the stabilizers to correct the side effects of projective measurements. 

Measuring in the Z-basis (computational basis) is equivalent to cutting the node from the graph and if the measurement outcome is 1, then a Z gate gets applied to all of its neighbors (nodes it had a cz gate with) in the remaining unmeasured part of the cluster state. 

Measuring in the X-basis (i.e applying a Hadamard gate before measuring in the computational basis) gives an interesting effect that can be illustrated as follows:

When measuring a node $n_i$, if measurement outcome of $n_i =1$:
- Apply X correction to the successor (according to flow, usually the node in the next layer on the same row)
- Apply Z correction to all neighbors of the successor (excluding $n_i$)

if measurement outcome of $n_i = 0$ we do nothing.

Another way of saying this is: In MBQC (initialized with $|+\rangle$ state) we always want to obtain a 0, if we do not get a zero, it means an error occurred and a correction needs to follow.

An illustration of this on a 2D cluster, when we measure qubit 1 and get 1:

![Byproduct correction illustration for 1st qubit](images/measure-1.png)

Then we measure qubit 2 and get 1:

![Byproduct correction illustration for 2nd qubit](images/measure-2.png)

This simple flow we used here works fine for brickwork and cluster states, for other irregular graph shapes, if a flow exists, there are ways to find it and find the order of measurements and corrections, but it is beyond the scope of this guide.

For the linear cluster example, we are trying to implement:

![Byproduct correction illustration for linear cluster](images/linear-correction.png)



## Implementation of measurement

First of all, we are going to define a function for X measurement, also instead of using the usual Qiskit notation with MSB first, we are going to use LSB first for the sake of easier readability and comparison with the visual cluster

In [1]:
# Linear cluster State preparation from the previous step
def x_measurement(qbit:int):
    # The function measures the LSB as the bit on the left (for consistency with the literature, as opposed to the usual order in Qiskit)
    # Measurement is done in the X basis, i.e Hadamard before measurement
    qc.h(qbit)
    qc.measure(q[qbit],c[nb_reg-1-qbit])

And here is what it looks like if we perform the measurement WITHOUT the byproduct correction

In [ ]:
x_measurement(0)
x_measurement(1)
x_measurement(2)
# then run it on a backend of choice.

If we run the above code we will get one of the following values:

- 11**1**
- 010
- 10**1**
- 000

But, we started our input qubit (the first qubit) in $|+\rangle$ that is $H|0\rangle$, so we expect to always find 0 for the last qubit (output qubit), but 2 of the possible outcomes are wrong, they are not performing the teleportation properly, we expect to always find 0 in the end, not 1. Which is why the byproduct correction conditioned by values of previous measurement outcome is important. So the final code after adding the correction becomes

In [3]:
from Qiskit import *

nb_reg=3
q = QuantumRegister(nb_reg)
c = ClassicalRegister(nb_reg) # Changed from 9 to 1
qc = QuantumCircuit(q, c)
# prepare |+++> state
for i in range(nb_reg):
    qc.h(i)
# prepare CZ(1,2)CZ(2,3)|+++> state
for i in range(nb_reg-1):
    qc.cz(i,i+1)

##########Measurement
def x_measurement(qbit:int):
    # The function measures the LSB as the bit on the left (for consistency with the literature, as opposed to the usual order in Qiskit)
    # Measurement is done in the X basis, i.e Hadamard before measurement
    qc.h(qbit)
    qc.measure(q[qbit],c[nb_reg-1-qbit])
    
x_measurement(0)

#qubit 0 will give X correction if =1 to qubit 1
with qc.if_test((c[nb_reg-1-(0)], 1)):
    qc.x(1)        
x_measurement(1)


#qubit 0 will give Z correction if =1 to qubit 2
with qc.if_test((c[nb_reg-1-(0)], 1)):
    qc.z(2)        
#qubit 1 will give X correction if =1 to qubit 2
with qc.if_test((c[nb_reg-1-(1)], 1)):
    qc.x(2)        
x_measurement(2)

print(qc.draw())

### Running the simulator to obtain measurement outcome
simulator = Aer.get_backend('aer_simulator_statevector') #qasm simulator
transpiled_circuit = transpile(qc, simulator)
job = execute(qc,simulator,shots = 10000)
result = job.result()
counts = result.get_counts(qc)
print(counts)


       ┌───┐   ┌───┐┌─┐                                               
q21_0: ┤ H ├─■─┤ H ├┤M├───────────────────────────────────────────────
       ├───┤ │ └───┘└╥┘┌──────────┐   ┌───┐    ┌─┐                    
q21_1: ┤ H ├─■───■───╫─┤0         ├───┤ H ├────┤M├────────────────────
       ├───┤     │   ║ │          │┌──┴───┴───┐└╥┘┌──────────┐┌───┐┌─┐
q21_2: ┤ H ├─────■───╫─┤          ├┤0         ├─╫─┤0         ├┤ H ├┤M├
       └───┘         ║ │          ││  If_else │ ║ │          │└───┘└╥┘
 c1_0: ══════════════╬═╡  If_else ╞╡          ╞═╬═╡          ╞══════╩═
                     ║ │          ││          │ ║ │          │        
 c1_1: ══════════════╬═╡          ╞╡          ╞═╩═╡0         ╞════════
                     ║ │          ││          │   └──────────┘        
 c1_2: ══════════════╩═╡0         ╞╡0         ╞═══════════════════════
                       └──────────┘└──────────┘                       
{'100': 2537, '000': 2494, '110': 2562, '010': 2407}


And here we can see that the last qubit is always = 0.

If we start in $|1\rangle$ instead of 0 before preparing the cluster, we should expect to get one in the last qubit.
The way to achieve this, is by simply applying an X gate in the code above before the preparation of the $|+++\rangle$ state so it becomes $|-++\rangle$.


In [4]:
from Qiskit import *

nb_reg=3
q = QuantumRegister(nb_reg)
c = ClassicalRegister(nb_reg) # Changed from 9 to 1
qc = QuantumCircuit(q, c)
#APPLY X GATE on first qubit
qc.x(0)

# prepare |-++> state
for i in range(nb_reg):
    qc.h(i)
# prepare CZ(1,2)CZ(2,3)|-++> state
for i in range(nb_reg-1):
    qc.cz(i,i+1)

##########Measurement
def x_measurement(qbit:int):
    # the function measures the MSB as the bit on the left (for consistency with the literature, as opposed to the usual order in Qiskit)
    # Measurement is done in the X basis, i.e Hadamard before measurement
    qc.h(qbit)
    qc.measure(q[qbit],c[nb_reg-1-qbit])
    
x_measurement(0)

#qubit 0 will give X correction if =1 to qubit 1
with qc.if_test((c[nb_reg-1-(0)], 1)):
    qc.x(1)        
x_measurement(1)


#qubit 0 will give Z correction if =1 to qubit 2
with qc.if_test((c[nb_reg-1-(0)], 1)):
    qc.z(2)        
#qubit 1 will give X correction if =1 to qubit 2
with qc.if_test((c[nb_reg-1-(1)], 1)):
    qc.x(2)        
x_measurement(2)

print(qc.draw())

### Running the simulator to obtain measurement outcome
simulator = Aer.get_backend('aer_simulator_statevector') #qasm simulator
transpiled_circuit = transpile(qc, simulator)
job = execute(qc,simulator,shots = 10000)
result = job.result()
counts = result.get_counts(qc)
print(counts)


       ┌───┐┌───┐   ┌───┐┌─┐                                               
q42_0: ┤ X ├┤ H ├─■─┤ H ├┤M├───────────────────────────────────────────────
       ├───┤└───┘ │ └───┘└╥┘┌──────────┐   ┌───┐    ┌─┐                    
q42_1: ┤ H ├──────■───■───╫─┤0         ├───┤ H ├────┤M├────────────────────
       ├───┤          │   ║ │          │┌──┴───┴───┐└╥┘┌──────────┐┌───┐┌─┐
q42_2: ┤ H ├──────────■───╫─┤          ├┤0         ├─╫─┤0         ├┤ H ├┤M├
       └───┘              ║ │          ││  If_else │ ║ │          │└───┘└╥┘
 c2_0: ═══════════════════╬═╡  If_else ╞╡          ╞═╬═╡          ╞══════╩═
                          ║ │          ││          │ ║ │          │        
 c2_1: ═══════════════════╬═╡          ╞╡          ╞═╩═╡0         ╞════════
                          ║ │          ││          │   └──────────┘        
 c2_2: ═══════════════════╩═╡0         ╞╡0         ╞═══════════════════════
                            └──────────┘└──────────┘                       
{'001': 2524

And now if we look, we will notice that we always get 1 in the final layer (the value of the last qubit), which means we teleported the 1 from the input layer successfully!


Two last remarks:
- The X correction in this particular example has no effect at all because we apply no phase shifts before measurements, therefore the state won't be affected since $X|+\rangle = |+\rangle$ so we may omit it
- Of course, If you wish to implement this with the Graphstate primitive provided by Qiskit that we quickly explored in the first section, you just need to replace `qc` by your graphstate (which we called `circ` in the last section), so the code (after removing the unnecessary x corrections) is:

In [5]:
import Qiskit as qs
mat=[[0,1,0],[1,0,1],[0,1,0]]
nb_reg=3
#this will prepare CZ(1,2) CZ(2,3) |+++> directly
circ=qs.circuit.library.GraphState(mat) 
cbits=ClassicalRegister(nb_reg)
circ.add_bits(cbits)


def x_measurement_g(qbit:int):
    # this function has been adapted  it is not exactly like the previous one
    # the function measures the MSB as the bit on the left (for consistency with the literature, as opposed to the usual order in Qiskit)
    # Measurement is done in the X basis, i.e Hadamard before measurement
    circ.h(qbit)
    circ.measure(qbit,(nb_reg-1-qbit))


x_measurement_g(0)
x_measurement_g(1)

with circ.if_test((cbits[2], 1)):
    circ.z(2)

x_measurement_g(2)

simulator = Aer.get_backend('aer_simulator_statevector') #qasm simulator
transpiled_circuit = transpile(circ, simulator)
job = execute(circ,simulator,shots = 100)
result = job.result()
counts = result.get_counts(circ)
print(counts)


{'010': 25, '000': 29, '110': 24, '100': 22}


Now before we conclude this chapter, let's have a quick look at more elaborate examples of computations in MBQC from [2]:


![Few examples of circuit equivalences in MBQC](images/mbqc-equiv.png)

Among these, we will focus on the Hadamard example. It can be also implemented as a linear cluster of 5 layers $\times$ 1 row only, we will ignore the second row of their brickwork state for simplicity, just like we implemented the identity on a liner cluster. The angles inside the circles mean that we apply a phase shift of the amount indicated inside the circle before the X measurement.


So the code for a Hadamard operation in MBQC inspired from the above is:

In [58]:
import Qiskit as qs
import numpy as np
mat=[[0,1,0,0,0],[1,0,1,0,0],[0,1,0,1,0],[0,0,1,0,1],[0,0,0,1,0]]
nb_reg=5
#this will prepare CZ(1,2) CZ(2,3)CZ(3,4),CZ(4,5) |+++++> directly
circ=qs.circuit.library.GraphState(mat) 
cbits=ClassicalRegister(nb_reg)
circ.add_bits(cbits)
angles=[np.pi/4,np.pi/4,np.pi/4,0,0]

def x_measurement_g(qbit:int):
    # This function has been adapted  it is not exactly like the previous one
    # The function measures the MSB as the bit on the left (for consistency with the literature, as opposed to the usual order in Qiskit)
    # Measurement is done in the X basis, i.e Hadamard before measurement
    # but before measurement we apply the phase shift
    circ.p(angles[qbit], qbit)
    circ.h(qbit)
    circ.measure(qbit,(nb_reg-1-qbit))

# This can be replaced by a more elegant loop, left as an exercise
x_measurement_g(0)
with circ.if_test((cbits[4-0], 1)):
    circ.x(1) # x on successor
    circ.z(2) # Z on neighbors of successor

x_measurement_g(1)
with circ.if_test((cbits[4-1], 1)):
    circ.x(2)
    circ.z(3)


x_measurement_g(2)
with circ.if_test((cbits[4-2], 1)):
    circ.x(3)
    circ.z(4)

x_measurement_g(3)
with circ.if_test((cbits[4-3], 1)):
    circ.x(4)

x_measurement_g(4)

circ.draw(output="mpl")
simulator = Aer.get_backend('aer_simulator_statevector') #qasm simulator
transpiled_circuit = transpile(circ, simulator)
job = execute(circ,simulator,shots = 10000)
result = job.result()
counts = result.get_counts(circ)
## we only care about the last layer/the last qubit, so we will filter the shots to get last value
output=dict()
output["0"]=0
output["1"]=0
for c,v in counts.items():
    output[c[-1]]+=v #get last bit from the bitstring
print(output)

{'0': 5691, '1': 4309}


As we can see we get almost a 50/50 chance of getting 0 or 1, which means it is an equal superposition of both, a Hadamard gate!

## references 

[1] A One-Way Quantum Computer, Robert Raussendorf and Hans J. Briegel, Phys. Rev. Lett. 86, 5188–5191 (2001)

[2] Garbled Quantum Computation, Elham Kashefi, and Petros Wallden, arXiv:1606.06931 (2016)

[3] Generalized Flow and Determinism in Measurement-based Quantum Computation, Browne, D. E., Kashefi, E., Mhalla, M., & Perdrix, S., New Journal of Physics 9, 250 (2007)

[4] Universality of quantum computation with cluster states and (X,Y)-plane measurements, A. Mantri, T. F. Demarie, J. F. Fitzsimons, arXiv:1607.00758

